In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from scipy.linalg import sqrtm
import os


def load_inception_model():
    print("Loading InceptionV3 model...")
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
    print("InceptionV3 model loaded.")
    return model


def preprocess_batch(images):
    print("Preprocessing batch of images...")
    images_resized = tf.image.resize(images, (299, 299)) 
    images_preprocessed = preprocess_input(images_resized) 
    print(f"Batch of {len(images)} images preprocessed.")
    return images_preprocessed

def calculate_statistics(images, inception_model, batch_size=32):
    print(f"Calculating statistics for {len(images)} images...")
    features = []
    for i in range(0, len(images), batch_size):
        batch = images[i:i+batch_size]
        batch_preprocessed = preprocess_batch(batch)
        batch_features = inception_model(batch_preprocessed, training=False).numpy()
        features.append(batch_features)
    features = np.concatenate(features, axis=0)
    mu = np.mean(features, axis=0)
    sigma = np.cov(features, rowvar=False)
    print("Statistics calculated (mean and covariance).")
    return mu, sigma


def calculate_fid(mu_real, sigma_real, mu_generated, sigma_generated, eps=1e-6):
    print("Calculating FID score...")
    diff = mu_real - mu_generated
    cov_sqrt, _ = sqrtm(sigma_real @ sigma_generated, disp=False)
    if np.iscomplexobj(cov_sqrt):
        cov_sqrt = cov_sqrt.real
    fid = diff @ diff + np.trace(sigma_real + sigma_generated - 2 * cov_sqrt)
    print(f"FID score calculated: {fid}")
    return fid

def generate_fake_images(generator, latent_dim, num_samples, batch_size=32):
    print(f"Generating {num_samples} fake images...")
    fake_images = []
    for _ in range(0, num_samples, batch_size):
        z = tf.random.normal([batch_size, latent_dim]) 
        batch_fake_images = generator(z, training=False)  
        batch_fake_images = (batch_fake_images + 1) / 2 
        fake_images.extend(batch_fake_images.numpy())
    print(f"{len(fake_images)} fake images generated.")
    return np.array(fake_images)


def compute_fid(real_images, generator, latent_dim, batch_size=32, num_samples=1000):
    print("Starting FID calculation...")
 
    inception_model = load_inception_model()

  
    print("Calculating statistics for real images...")
    mu_real, sigma_real = calculate_statistics(real_images, inception_model, batch_size)

 
    print("Calculating statistics for generated images...")
    generated_images = generate_fake_images(generator, latent_dim, num_samples, batch_size)
    mu_generated, sigma_generated = calculate_statistics(generated_images, inception_model, batch_size)

  
    fid_score = calculate_fid(mu_real, sigma_real, mu_generated, sigma_generated)
    print(f"FID score: {fid_score}")
    return fid_score


if __name__ == "__main__":
   
    SIZE = 128 
    real_images = []
    path = "../input/face-mask-lite-dataset/without_mask" 
    files = sorted(os.listdir(path))  

    
    print("Loading real images...")
    for i, file in enumerate(files):
        if i == 200:  
            break
        if file == "seed9090.png":
            continue
        img = tf.image.decode_image(tf.io.read_file(os.path.join(path, file)))
        img = tf.image.resize(img, (SIZE, SIZE))
        img = (img - 127.5) / 127.5 
        real_images.append(img)
    real_images = tf.convert_to_tensor(real_images, dtype=tf.float32)
    print(f"Loaded {len(real_images)} real images.")

   
    generator_path = "/kaggle/input/1/other/default/1"  
    print("Loading generator model...")
    generator = tf.saved_model.load(generator_path)
    print("Generator model loaded.")

    
    latent_dim = 100  
    fid_score = compute_fid(real_images, generator, latent_dim, batch_size=32, num_samples=200)

    print(f"Final FID Score: {fid_score}")

Loading real images...
Loaded 200 real images.
Loading generator model...
Generator model loaded.
Starting FID calculation...
Loading InceptionV3 model...
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
InceptionV3 model loaded.
Calculating statistics for real images...
Calculating statistics for 200 images...
Preprocessing batch of images...
Batch of 32 images preprocessed.
Preprocessing batch of images...
Batch of 32 images preprocessed.
Preprocessing batch of images...
Batch of 32 images preprocessed.
Preprocessing batch of images...
Batch of 32 images preprocessed.
Preprocessing batch of images...
Batch of 32 images preprocessed.
Preprocessing batch of images...
Batch of 32 images preprocessed.
Preprocessing batch of images...
Batch of 8 images preprocessed.
Statistics calculated (mean and covariance).
Calculating statistics for generated images...
Generating 200 fake images...
224 fake images generated.
Calculating statistics for 224 images...
Preprocessing batch of images...
B